# 1. Obtención de datos

In [ ]:
! wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
! tar -xf aclImdb_v1.tar.gz

In [ ]:
!pip install -r requirements.txt

# 2. Exploración de datos

In [ ]:
import os
import glob

def read_imdb_data(data_dir='./aclImdb'):
    """Read IMDb movie reviews from given directory.
    
    Directory structure expected:
    - data/
        - train/
            - pos/
            - neg/
        - test/
            - pos/
            - neg/
    
    """

    # Datos y etiquetas que se devolveran en dict anidades replicando la estructura de directorios
    data = {}
    labels = {}

    # Asumimos 2 subdirectorios: train y test
    for data_type in ['train', 'test']:
        data[data_type] = {}
        labels[data_type] = {}

        # Asumimos 2 subdirectorios por sentimmiento (etiqueta): pos, neg
        for sentiment in ['pos', 'neg']:
            data[data_type][sentiment] = []
            labels[data_type][sentiment] = []
            
            # Recogemos los ficheros por directorio
            path = os.path.join(data_dir, data_type, sentiment, '*.txt')
            files = glob.glob(path)
            
            # Leemos los datos y les asignamos a su etiqueta
            for f in files:
                with open(f) as review:
                    data[data_type][sentiment].append(review.read())
                    labels[data_type][sentiment].append(sentiment)
            
            assert len(data[data_type][sentiment]) == len(labels[data_type][sentiment]), \
                    "{}/{} data size does not match labels size".format(data_type, sentiment)
    
    # Devolvemos los datos y etiquetas en diccionarios anidados
    return data, labels


data, labels = read_imdb_data()
print("IMDb reviews: train = {} pos / {} neg, test = {} pos / {} neg".format(
        len(data['train']['pos']), len(data['train']['neg']),
        len(data['test']['pos']), len(data['test']['neg'])))

IMDb reviews: train = 12500 pos / 12500 neg, test = 12500 pos / 12500 neg


In [ ]:
data["train"]["pos"][0]

In [ ]:
data["train"]["neg"][0]

In [ ]:
!pip install wordcloud

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

from wordcloud import WordCloud, STOPWORDS

sentiment = 'pos'

# Combinamos las reviews para el sentimiento deseado
combined_text = " ".join([review for review in data['train'][sentiment]])

# Inicializamos el wordcloud
wc = WordCloud(background_color='white', max_words=50,
        # Actualizamos las stopwords para incluir palabras comunes del tema
        stopwords = STOPWORDS.update(['br','film','movie']))

plt.imshow(wc.generate(combined_text))
plt.axis('off')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

from wordcloud import WordCloud, STOPWORDS

sentiment = 'neg'

combined_text = " ".join([review for review in data['train'][sentiment]])

wc = WordCloud(background_color='white', max_words=50,
        stopwords = STOPWORDS.update(['br','film','movie']))

plt.imshow(wc.generate(combined_text))
plt.axis('off')
plt.show()

In [ ]:
from sklearn.utils import shuffle

def prepare_imdb_data(data):
    """Prepare training and test sets from IMDb movie reviews."""
    
    # Combinamos los datos de las diferentes etiquetas
    data_train = data['train']['pos'] + data['train']['neg']
    data_test = data['test']['pos'] + data['test']['neg']
    labels_train = labels['train']['pos'] + labels['train']['neg']
    labels_test = labels['test']['pos'] + labels['test']['neg']
    
    # Mezclamos los datos con sus etiquetas
    data_train, labels_train = shuffle(data_train, labels_train)
    data_test, labels_test = shuffle(data_test, labels_test)
    
    # Devolvemos unificado training data, test data, training labels, test labets
    return data_train, data_test, labels_train, labels_test


data_train, data_test, labels_train, labels_test = prepare_imdb_data(data)
print("IMDb reviews (combined): train = {}, test = {}".format(len(data_train), len(data_test)))

IMDb reviews (combined): train = 25000, test = 25000


# 4. Preprocesamiento

In [ ]:
# BeautifulSoup to easily remove HTML tags
from bs4 import BeautifulSoup 

# RegEx for removing non-letter characters
import re

# NLTK library for the remaining steps
import nltk
nltk.download("stopwords")   # download list of stopwords (only once; need not run it again)
from nltk.corpus import stopwords # import stopwords

from nltk.stem.porter import *
stemmer = PorterStemmer()

In [ ]:
def review_to_words(review):
    """Convert a raw review string into a sequence of words."""
    
    # Eliminamos las etiquetas HTML
    text = BeautifulSoup(review, "html5lib").get_text()
    # Convertimos a minúscula y quitamos todo lo que no sea texto o números
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    # Dividimos en tokens por espacios
    words = text.split()
    # Eliminamos stopwords
    words = [w for w in words if w not in stopwords.words("english")]
    # Aplicamos stemming
    words = [PorterStemmer().stem(w) for w in words]

    return words


review_to_words("""This is just a <em>test</em>.<br/><br />
But if it wasn't a test, it would make for a <b>Great</b> movie review!""")

In [ ]:
import pickle

cache_dir = os.path.join("cache", "sentiment_analysis")
os.makedirs(cache_dir, exist_ok=True)

def preprocess_data(data_train, data_test, labels_train, labels_test,
                    cache_dir=cache_dir, cache_file="preprocessed_data.pkl"):
    """Convert each review to words; read from cache if available."""

    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = pickle.load(f)
            print("Read preprocessed data from cache file:", cache_file)
        except:
            pass
    
    if cache_data is None:
        words_train = list(map(review_to_words, data_train))
        words_test = list(map(review_to_words, data_test))
        
        if cache_file is not None:
            cache_data = dict(words_train=words_train, words_test=words_test,
                              labels_train=labels_train, labels_test=labels_test)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file:", cache_file)
    else:
        words_train, words_test, labels_train, labels_test = (cache_data['words_train'],
                cache_data['words_test'], cache_data['labels_train'], cache_data['labels_test'])
    
    return words_train, words_test, labels_train, labels_test


words_train, words_test, labels_train, labels_test = preprocess_data(
        data_train, data_test, labels_train, labels_test)

print("\n--- Raw review ---")
print(data_train[1])
print("\n--- Preprocessed words ---")
print(words_train[1])
print("\n--- Label ---")
print(labels_train[1])

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import joblib

def extract_BoW_features(words_train, words_test, vocabulary_size=5000,
                         cache_dir=cache_dir, cache_file="bow_features.pkl"):
    """Extract Bag-of-Words for a given set of documents, already preprocessed into words."""
    
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = joblib.load(f)
            print("Read features from cache file:", cache_file)
        except:
            pass
    
    if cache_data is None:
        vectorizer = CountVectorizer(max_features=vocabulary_size,
                preprocessor=lambda x: x, tokenizer=lambda x: x)  # already preprocessed
        features_train = vectorizer.fit_transform(words_train).toarray()

        features_test = vectorizer.transform(words_test).toarray()
                
        if cache_file is not None:
            vocabulary = vectorizer.vocabulary_
            cache_data = dict(features_train=features_train, features_test=features_test,
                             vocabulary=vocabulary)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                joblib.dump(cache_data, f)
            print("Wrote features to cache file:", cache_file)
    else:
        features_train, features_test, vocabulary = (cache_data['features_train'],
                cache_data['features_test'], cache_data['vocabulary'])
    
    return features_train, features_test, vocabulary

features_train, features_test, vocabulary = extract_BoW_features(words_train, words_test)

print("Vocabulary: {} words".format(len(vocabulary)))

import random
print("Sample words: {}".format(random.sample(list(vocabulary.keys()), 8)))

print("\n--- Preprocessed words ---")
print(words_train[5])
print("\n--- Bag-of-Words features ---")
print(features_train[5])
print("\n--- Label ---")
print(labels_train[5])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#Frecuencia de las palabras. Observamos algunos outliers. 
#Esos datos tienen q ser significativos dado que se ha realizado un preprocesamiento previo
plt.plot(features_train[5,:])
plt.xlabel('Word')
plt.ylabel('Count')
plt.show()

In [ ]:
#Observamos la ley de Zipf
word_freq = features_train.sum(axis=0)

sorted_word_freq = np.sort(word_freq)[::-1]

plt.plot(sorted_word_freq)
plt.gca().set_xscale('log')
plt.gca().set_yscale('log')
plt.xlabel('Rank')
plt.ylabel('Number of occurrences')
plt.show()

In [ ]:
import sklearn.preprocessing as pr

features_train = pr.normalize(features_train, axis=1)
features_test = pr.normalize(features_test, axis=1)

Debate: ¿Por qué normalizamos los BoW?

# 4. Entrenamiento

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

n_estimators = 32

def classify_gboost(X_train, X_test, y_train, y_test):        
    clf = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=1.0, max_depth=1, random_state=42)

    clf.fit(X_train, y_train)
    
    print("[{}] Accuracy: train = {}, test = {}".format(
            clf.__class__.__name__,
            clf.score(X_train, y_train),
            clf.score(X_test, y_test)))
    
    return clf


clf2 = classify_gboost(features_train, features_test, labels_train, labels_test)

# 5. Pasamos a Deep Learning!

In [ ]:
from keras.datasets import imdb

vocabulary_size = 5000

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocabulary_size)
print("Loaded dataset with {} training samples, {} test samples".format(len(X_train), len(X_test)))

In [ ]:
(X_train_example, y_train_example), (X_test_example, y_test_example) = imdb.load_data(num_words=vocabulary_size)

In [ ]:
print("--- Review ---")
print(X_train[0])
print("--- Label ---")
print(y_train[0])

In [ ]:
# Datos ya vienen tokenizados (no son palabras, asignados a un identificador)

In [ ]:
#cambiamos de número a palabras (metodo de Keras)
word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}
print("--- Review (with words) ---")
print([id2word.get(i, " ") for i in X_train[0]])
print("--- Label ---")
print(y_train[0])

In [ ]:
# Media de palabras en la reviews.
# RRN necesitamos establcer el tamaño de neruonas 
# primero hacer el análisis para ver el número medio y máxcimo de palabras en las revies.
#si una frase se queda corta, la palabrs restantes se rellenaran de 0. Esto sería negativop
max = 0
mean = []
for example in X_train_example:
  length = len(example)
  mean.append(len(example))
  if length > max:
    max = length

In [ ]:
sum(mean) / len(mean)

In [ ]:
max

In [ ]:
from tensorflow.keras.utils import pad_sequences

max_words = 500

X_train = pad_sequences(X_train, maxlen=max_words)
X_test = pad_sequences(X_test, maxlen=max_words)

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, GRUV2, SimpleRNN

embedding_size = 32
model = Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

print(model.summary())

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
batch_size = 64
num_epochs = 1

X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]  # first batch_size samples
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]  # rest for training

model.fit(X_train2, y_train2,
          validation_data=(X_valid, y_valid),
          batch_size=batch_size, epochs=num_epochs)

In [ ]:
import os


In [ ]:
model_file = "lstm_model.h5"  # HDF5 file
model.save(os.path.join(cache_dir, model_file))

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)  # returns loss and other metrics specified in model.compile()
print("Test accuracy:", scores[1])  # scores[1] should correspond to accuracy if you passed in metrics=['accuracy']

Vamos a ver con GRU:

In [ ]:
embedding_size = 32
model_gru = Sequential()
model_gru.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
model_gru.add(GRUV2(100))
model_gru.add(Dense(1, activation='sigmoid'))

print(model_gru.summary())

In [ ]:
model_gru.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
batch_size = 64
num_epochs = 1

X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]  # first batch_size samples
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]  # rest for training

model_gru.fit(X_train2, y_train2,
          validation_data=(X_valid, y_valid),
          batch_size=batch_size, epochs=num_epochs)

In [ ]:
model_file = "gru_model.h5"  # HDF5 file
model_gru.save(os.path.join(cache_dir, model_file))

In [ ]:
scores = model_gru.evaluate(X_test, y_test, verbose=0)  # returns loss and other metrics specified in model.compile()
print("Test accuracy:", scores[1])  # scores[1] should correspond to accuracy if you passed in metrics=['accuracy']

Y por último con RNN:

In [ ]:
embedding_size = 32
model_rnn = Sequential()
model_rnn.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
model_rnn.add(SimpleRNN(100))
model_rnn.add(Dense(1, activation='sigmoid'))

print(model_rnn.summary())

In [ ]:
model_rnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
batch_size = 64
num_epochs = 1

X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]  # first batch_size samples
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]  # rest for training

model_rnn.fit(X_train2, y_train2,
          validation_data=(X_valid, y_valid),
          batch_size=batch_size, epochs=num_epochs)

In [ ]:
model_file = "rnn_model.h5"  # HDF5 file
model_rnn.save(os.path.join(cache_dir, model_file))

In [ ]:
scores = model_rnn.evaluate(X_test, y_test, verbose=0)  # returns loss and other metrics specified in model.compile()
print("Test accuracy:", scores[1])  # scores[1] should correspond to accuracy if you passed in metrics=['accuracy']

Y si quitamos el Embedding?

# 5. RNN + word2vec Embedding

In [ ]:
import gensim
import multiprocessing as mp

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense,
    Dropout,
    Embedding,
    LSTM,
)
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.preprocessing import LabelEncoder
 

In [ ]:
# Parámetros del WORD2VEC
W2V_SIZE = 300 # tamaño de vectores
W2V_WINDOW = 7 # número de palabras que va a mirar alrededor
# 32
W2V_EPOCH = 5 # número de epoca
W2V_MIN_COUNT = 2 #número mínimo de frecuencia

# KERAS
SEQUENCE_LENGTH = 500 # número de secuencias de keras

In [ ]:
def generate_tokenizer(train_df):
  #generamos un identificador único para cada una de las palabras
  #creamos el vocabulario (tenemos palabra y número)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(train_df)
    vocab_size = len(tokenizer.word_index) + 1
    print(f"Total words: {vocab_size}")
    return tokenizer, vocab_size

In [ ]:
def generate_word2vec(train_df):
  #para cada review la dividimos en tokens (palabras )
  #para todos los documentos tiene todas las palabras generamos un identificados único para cada una de las palabras
  #generamos el modelo, utilizamos la librería gemsin con los parametros definidos previamente, vocabulario y luego entrenamiento
  #creamos representacion vecrtoriales (en arrays de 300 elementos)
    documents = [_text.split() for _text in train_df.review]
    w2v_model = gensim.models.word2vec.Word2Vec(
        size=W2V_SIZE,
        window=W2V_WINDOW,
        min_count=W2V_MIN_COUNT,
        workers=mp.cpu_count(),
    )
    w2v_model.build_vocab(documents)

    words = w2v_model.wv.vocab.keys()
    vocab_size = len(words)
    print(f"Vocab size: {vocab_size}")
    w2v_model.train(documents, total_examples=len(documents), epochs=W2V_EPOCH)

    return w2v_model

In [ ]:
# Generamos la capa de embeddings
# El modelo no tiene que hacer el fit de la capa de embeddings, se la damos dada

def generate_embedding(word2vec_model, vocab_size, tokenizer):
  # generamos un capa de embedding como en el ejemplo anterior con LSTM y GRU pero
  # en este caso lo hacemos por separado y guardamos el objeto
  # Generamos la matriz de embedding, inicializamos a 0 con el tamaño de  W2V y vocabulario
  # Para cada palabra del vocabulario generamos la representación vectorial de w2v
  # y se la asignamos a la matriz. De este modo, generamos la matrix de equivalencia
    embedding_matrix = np.zeros((vocab_size, W2V_SIZE))
    for word, i in tokenizer.word_index.items():
        if word in word2vec_model.wv:
            embedding_matrix[i] = word2vec_model.wv[word]
    return Embedding(
        vocab_size,
        W2V_SIZE,
        weights=[embedding_matrix],
        input_length=SEQUENCE_LENGTH,
        # Con trainable le indicamos que no se modificar lo que ya viene dentro de esa capa 
        trainable=False,
    )

    # Devuelve la capa de embedding

In [ ]:
#cargamos los datos con imdb. Tendremos el vocabularion con los índices 
(X_train_index, y_train_index), (X_test_index, y_test_index) = imdb.load_data(num_words=vocabulary_size)

In [ ]:
#convertimos a palabras. Una lista con cada review (un string único)
word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}

X_train_words = []
for index, example in enumerate(X_train_index):
  words = " ".join([id2word.get(i, " ") for i in X_train_index[index]]).strip()
  X_train_words.append(words)

In [ ]:
X_test_words = []
for index, example in enumerate(X_test_index):
  words = " ".join([id2word.get(i, " ") for i in X_test_index[index]]).strip()
  X_test_words.append(words)

In [ ]:
X_train_words[10]

In [ ]:

import pandas as pd

X_train_words = pd.DataFrame(X_train_words, columns=["review"])
X_test_words = pd.DataFrame(X_test_words, columns=["review"])

In [ ]:
X_train_words.head()

In [ ]:
tokenizer, vocab = generate_tokenizer(X_train_words.review)

In [ ]:
word2vec_model = generate_word2vec(X_train_words)

In [ ]:
from tensorflow.keras.utils import pad_sequences


max_words = 500

#Pasamos el pading (500 palabras), tokenizamos pero antes pasamos el texto a secuencias 
#Cuando tokenizamos pasamos todas las palabras y se convierte a un identificor único del vocabulario 

X_train_words = pad_sequences(tokenizer.texts_to_sequences(X_train_words.review), maxlen=max_words)
X_test_words = pad_sequences(tokenizer.texts_to_sequences(X_test_words.review), maxlen=max_words)

In [ ]:
import numpy as np
embedding_layer = generate_embedding(word2vec_model, vocab, tokenizer)

In [ ]:
model_custom = Sequential()
#pasamos directamente la capa que hemos generado
model_custom.add(embedding_layer)
#dropout, función entre celdas y el recurrent_dropout añade aleatoriedad (afecta a las gates)
model_custom.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model_custom.add(Dense(1, activation="sigmoid"))

In [ ]:
model_custom.summary()

In [ ]:
model_custom.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
batch_size = 64
num_epochs = 1

X_train_words_valid, y_valid = X_train_words[:batch_size], y_train[:batch_size]  # first batch_size samples
X_train_words2, y_train2 = X_train_words[batch_size:], y_train[batch_size:]  # rest for training

model_custom.fit(X_train_words2, y_train2,
          validation_data=(X_train_words_valid, y_valid),
          batch_size=batch_size, epochs=num_epochs)